# Read Data

In [4]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('ATOM_USDT_futures_data_2020.csv')

# Drop unnecessary columns
df.drop(columns=(['coin','volume','closetime','quotevolume','trades','taker_buy_volume','taker_buy_quote','unused']), inplace=True)

# Display the first few rows of the DataFrame
df

,opentime,openprice,highprice,lowprice,closeprice
0,1581076800000,4.574,4.870,4.573,4.610
1,1581080400000,4.614,4.700,4.610,4.690
2,1581084000000,4.685,4.870,4.658,4.839
3,1581087600000,4.838,4.895,4.775,4.886
4,1581091200000,4.874,4.909,4.782,4.815
...,...,...,...,...,...
42553,1734267600000,9.054,9.212,9.000,9.172
42554,1734271200000,9.173,9.308,9.172,9.233
42555,1734274800000,9.233,9.255,9.133,9.190
42556,1734278400000,9.190,9.223,9.068,9.088


# Add Super Trend Indicator

In [9]:
import pandas as pd
from pandas_ta.overlap import hl2
from pandas_ta.volatility import atr
from pandas_ta.utils import get_offset, verify_series

# Apply the Supertrend function
def supertrend(high, low, close, length=7, multiplier=3.0, offset=0, **kwargs):
    """Indicator: Supertrend"""
    # Validate Arguments
    high = verify_series(high, length)
    low = verify_series(low, length)
    close = verify_series(close, length)
    offset = get_offset(offset)

    if high is None or low is None or close is None: return

    m = close.size
    dir_, trend = [1] * m, [0] * m
    long, short = [npNaN] * m, [npNaN] * m

    hl2_ = hl2(high, low)
    matr = multiplier * atr(high, low, close, length)
    upperband = hl2_ + matr
    lowerband = hl2_ - matr

    for i in range(1, m):
        if close.iloc[i] > upperband.iloc[i - 1]:
            dir_[i] = 1
        elif close.iloc[i] < lowerband.iloc[i - 1]:
            dir_[i] = -1
        else:
            dir_[i] = dir_[i - 1]
            if dir_[i] > 0 and lowerband.iloc[i] < lowerband.iloc[i - 1]:
                lowerband.iloc[i] = lowerband.iloc[i - 1]
            if dir_[i] < 0 and upperband.iloc[i] > upperband.iloc[i - 1]:
                upperband.iloc[i] = upperband.iloc[i - 1]

        if dir_[i] > 0:
            trend[i] = long[i] = lowerband.iloc[i]
        else:
            trend[i] = short[i] = upperband.iloc[i]

    # Prepare DataFrame to return
    _props = f"_{length}_{multiplier}"
    df = pd.DataFrame({
            f"SUPERT{_props}": trend,
            f"SUPERTd{_props}": dir_,
            f"SUPERTl{_props}": long,
            f"SUPERTs{_props}": short,
        }, index=close.index)

    df.name = f"SUPERT{_props}"
    df.category = "overlap"

    # Apply offset if needed
    if offset != 0:
        df = df.shift(offset)

    # Handle fills
    if "fillna" in kwargs:
        df.fillna(kwargs["fillna"], inplace=True)

    if "fill_method" in kwargs:
        df.fillna(method=kwargs["fill_method"], inplace=True)

    return df

In [11]:
# Get the Supertrend values using the 'highprice', 'lowprice', and 'closeprice' columns
supertrend_indicator = supertrend(df['highprice'], df['lowprice'], df['closeprice'], length=10, multiplier=3.0)

# Join the Supertrend indicator values to your original dataframe
supertrend_df = df.join(supertrend_indicator)

# Check the output
supertrend_df

,opentime,openprice,highprice,lowprice,closeprice,SUPERT_10_3.0,SUPERTd_10_3.0,SUPERTl_10_3.0,SUPERTs_10_3.0
0,1581076800000,4.574,4.870,4.573,4.610,0.000000,1,NaN,NaN
1,1581080400000,4.614,4.700,4.610,4.690,NaN,1,NaN,NaN
2,1581084000000,4.685,4.870,4.658,4.839,NaN,1,NaN,NaN
3,1581087600000,4.838,4.895,4.775,4.886,NaN,1,NaN,NaN
4,1581091200000,4.874,4.909,4.782,4.815,NaN,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...
42553,1734267600000,9.054,9.212,9.000,9.172,9.178569,-1,NaN,9.178569
42554,1734271200000,9.173,9.308,9.172,9.233,8.789368,1,8.789368,NaN
42555,1734274800000,9.233,9.255,9.133,9.190,8.789368,1,8.789368,NaN
42556,1734278400000,9.190,9.223,9.068,9.088,8.789368,1,8.789368,NaN
